In [43]:
import pandas as pd
data  = pd.read_csv('../content/drive/MyDrive/ML Project/Progresspulse Data.csv')

In [44]:
data.columns

Index(['Email Address', 'College Name', 'Department', 'Level Of Study',
       'Current year of study', 'CGPA', 'Credits', 'ExtraCurricular',
       'Projects', 'Self Study', 'Engagement', 'Assignment',
       'OutputPerformance', 'Contribution'],
      dtype='object')

In [45]:
k = pd.DataFrame()
k = data[['CGPA', 'Credits','ExtraCurricular', 'Projects', 'Self Study', 'Engagement', 'Assignment','Contribution','OutputPerformance']]
k.corr(method='pearson',min_periods=9)['OutputPerformance']

,OutputPerformance
CGPA,0.117173
Credits,-0.262921
ExtraCurricular,-0.038675
Projects,-0.081889
Self Study,0.111683
Engagement,0.696361
Assignment,0.433251
Contribution,0.211144
OutputPerformance,1.000000


In [46]:
feat = ['CGPA', 'Credits',
    'ExtraCurricular', 'Projects', 'Self Study', 'Engagement', 'Assignment',
    'Contribution']
X = data[feat]
y = data['OutputPerformance']
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y = train_test_split(X,y,test_size=0.48)
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error,confusion_matrix, make_scorer, accuracy_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier

In [47]:
print('sklearn',sklearn.__version__)
print('pandas',pd.__version__)
print('numpy',np.__version__)
print('seaborn',sns.__version__)

sklearn 1.6.1
pandas 2.2.2
numpy 2.0.2
seaborn 0.13.2


In [48]:
model = {'RandomForest Classifier' : RandomForestClassifier(max_depth=212,n_estimators=500),'Decision Tree Classifier' : DecisionTreeClassifier(splitter='best',random_state=1),'Logistic Regression' : LogisticRegression(random_state=289),'RandomForest Regressor' : RandomForestRegressor(criterion='absolute_error'),'Decision Tree Regressor' : DecisionTreeRegressor(random_state=289),'K Nearest Classifier' : KNeighborsClassifier(n_neighbors=11)}
for i in range(len(list(model))):
  mdls = list(model.values())[i]
  mdls.fit(train_X,train_y)
  train_pred_y = mdls.predict(train_X)
  pred_y = mdls.predict(val_X)
  me_err = mean_absolute_error(val_y,pred_y)
  print(list(model.keys())[i])
  print('mae ' ,me_err )
  print('-'*15)


RandomForest Classifier
mae  1.411764705882353
---------------
Decision Tree Classifier
mae  1.8823529411764706
---------------
Logistic Regression
mae  1.7058823529411764
---------------


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForest Regressor
mae  1.6370588235294117
---------------
Decision Tree Regressor
mae  1.8823529411764706
---------------
K Nearest Classifier
mae  2.176470588235294
---------------


In [49]:
param_space = {
    'C': (1e-6, 1e+6, 'log-uniform'),
    'gamma': (1e-6, 1e+1, 'log-uniform'),
    'degree': (1, 8),  # integer valued parameter
    'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
}

In [50]:
pip install scikit-optimize

In [51]:
from skopt import BayesSearchCV
from sklearn.svm import SVC
opt = BayesSearchCV(
    SVC(),
    param_space,
    n_iter=10
)

In [52]:
opt.fit(train_X, train_y)

# Get best hyperparameters
best_params = opt.best_params_


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklea

In [53]:
# Get best hyperparameters
best_params = opt.best_params_

# Create an SVM classifier with the best parameters
best_svc_model = SVC(**best_params)

# Fit the classifier on the training data
best_svc_model.fit(train_X, train_y)

# Predict on the test data
best_svc_pred = best_svc_model.predict(val_X)

# Evaluate the performance of the model
print("Train Accuracy with best parameters:", accuracy_score(train_y, best_svc_model.predict(train_X)))
print("Test Accuracy with best parameters: ", accuracy_score(val_y, best_svc_pred))

Train Accuracy with best parameters: 0.3333333333333333
Test Accuracy with best parameters:  0.23529411764705882


In [54]:
# mdl = RandomForestRegressor(criterion='friedman_mse')
mdl=best_svc_model.fit(train_X, train_y)

mdl.fit(train_X,train_y)
pred1_y = mdl.predict(val_X)


In [55]:
import pickle
with open("../trainmlx.pkl", "wb") as file:
  pickle.dump(mdl,file)